In [1]:
import duckdb
import requests
import json
import polars as pl
import requests
import logging
import time
import os
import psycopg2
from msgspec.json import decode
from msgspec import Struct
from typing import Optional
from datetime import datetime
from job_board_scraper.utils import general as util


In [2]:
connection = psycopg2.connect(
    host=os.environ.get("PG_HOST"),
    user=os.environ.get("PG_USER"),
    password=os.environ.get("PG_PASSWORD"),
    dbname=os.environ.get("PG_DATABASE"),
)
cursor = connection.cursor()
cursor.execute("""select * from public.job_board_urls where url like '%.ashbyhq.%'""")
careers_page_urls = cursor.fetchall()
cursor.close()
connection.close()

In [3]:
con = duckdb.connect(database=':memory:')
logger = logging.getLogger("ashby_logger")

In [4]:
careers_page_urls = [
"https://jobs.ashbyhq.com/StandardBots",
"https://jobs.ashbyhq.com/linear",
"https://jobs.ashbyhq.com/Pulley",
"https://jobs.ashbyhq.com/trucksmarter",
"https://jobs.ashbyhq.com/anrok",
"https://jobs.ashbyhq.com/slope",
"https://jobs.ashbyhq.com/convex-dev",
"https://jobs.ashbyhq.com/MotherDuck",
"https://jobs.ashbyhq.com/Crusoe",
"https://jobs.ashbyhq.com/found",
"https://jobs.ashbyhq.com/sardine",
"https://jobs.ashbyhq.com/nirvana",
"https://jobs.ashbyhq.com/character",
"https://jobs.ashbyhq.com/sleeper",
"https://jobs.ashbyhq.com/pathmentalhealth",
"https://jobs.ashbyhq.com/formenergy",
"https://jobs.ashbyhq.com/vanta",
"https://jobs.ashbyhq.com/middesk",
"https://jobs.ashbyhq.com/ramp",
"https://jobs.ashbyhq.com/bigeye",
"https://jobs.ashbyhq.com/evenup",
"https://jobs.ashbyhq.com/openstore",
"https://jobs.ashbyhq.com/eightsleep",
"https://jobs.ashbyhq.com/replit",
"https://jobs.ashbyhq.com/virtahealth",
"https://jobs.ashbyhq.com/shiftsmart",
"https://jobs.ashbyhq.com/Deel",
"https://jobs.ashbyhq.com/unit",
"https://jobs.ashbyhq.com/montecarlodata",
"https://jobs.ashbyhq.com/parafin",
"https://jobs.ashbyhq.com/aurorasolar",
]

In [5]:
careers_page_urls

['https://jobs.ashbyhq.com/StandardBots',
 'https://jobs.ashbyhq.com/linear',
 'https://jobs.ashbyhq.com/Pulley',
 'https://jobs.ashbyhq.com/trucksmarter',
 'https://jobs.ashbyhq.com/anrok',
 'https://jobs.ashbyhq.com/slope',
 'https://jobs.ashbyhq.com/convex-dev',
 'https://jobs.ashbyhq.com/MotherDuck',
 'https://jobs.ashbyhq.com/Crusoe',
 'https://jobs.ashbyhq.com/found',
 'https://jobs.ashbyhq.com/sardine',
 'https://jobs.ashbyhq.com/nirvana',
 'https://jobs.ashbyhq.com/character',
 'https://jobs.ashbyhq.com/sleeper',
 'https://jobs.ashbyhq.com/pathmentalhealth',
 'https://jobs.ashbyhq.com/formenergy',
 'https://jobs.ashbyhq.com/vanta',
 'https://jobs.ashbyhq.com/middesk',
 'https://jobs.ashbyhq.com/ramp',
 'https://jobs.ashbyhq.com/bigeye',
 'https://jobs.ashbyhq.com/evenup',
 'https://jobs.ashbyhq.com/openstore',
 'https://jobs.ashbyhq.com/eightsleep',
 'https://jobs.ashbyhq.com/replit',
 'https://jobs.ashbyhq.com/virtahealth',
 'https://jobs.ashbyhq.com/shiftsmart',
 'https://job

In [6]:
ashby_teams_id = 3
ashby_postings_id = 4
ashby_locations_id = 5

In [7]:
def determine_row_id(i):
    return util.hash_ids.encode(3, i, url_id, created_at)

In [26]:
class SecondaryLocation(Struct):
    locationId: str
    locationName: str

class Posting(Struct):
    id: str
    title: str
    teamId: str
    locationId: str
    locationName: str
    employmentType: str
    compensationTierSummary: Optional[str]
    secondaryLocations: Optional[list[SecondaryLocation]]

class Team(Struct):
    id: str
    name: str
    parentTeamId: Optional[str]

In [27]:
%%time
url = "https://jobs.ashbyhq.com/api/non-user-graphql?op=ApiJobBoardWithTeams"

f = open('queries/ashby_jobs_outline.graphql')
query = f.read()

headers = {
  'Content-Type': 'application/json'
}


for ashby_url in careers_page_urls:
  ashby_company = ashby_url.split('/')[-1]

  variables = {
    "organizationHostedJobsPageName": ashby_company
  }

  current_time = time.time()
  created_at = int(current_time)
  updated_at = int(current_time)
  current_date_utc = datetime.utcfromtimestamp(current_time).strftime("%Y-%m-%d")

  response = requests.request("POST", url, headers=headers, json={"query": query, "variables": variables})

  response_text = response.text
  print(response.text)
  assert response.status_code == 200
  response_json = json.loads(response_text)
  assert response_json['data']['jobBoard'] != None
  # break

{"data":{"jobBoard":{"teams":[{"id":"f4076691-49ea-4b27-8a63-4ff3afe6395c","name":"Operations","parentTeamId":null,"__typename":"Team"},{"id":"ace4109b-9d64-4a12-a3e4-bc1534794f30","name":"Software","parentTeamId":null,"__typename":"Team"}],"jobPostings":[{"id":"327e8df3-fa76-42c8-8fbf-58f11f3d3d3f","title":"Assembly Technician","teamId":"f4076691-49ea-4b27-8a63-4ff3afe6395c","locationId":"bce6339c-6b47-42c7-b2dd-8ef5234e4d31","locationName":"Glen Cove","employmentType":"FullTime","secondaryLocations":[],"compensationTierSummary":null,"__typename":"JobPostingBriefsWithIdsAndTeamId"},{"id":"697a5570-d6b4-4bee-9d41-d5e5dccc9f7b","title":"Senior Software Engineer, Full-Stack","teamId":"ace4109b-9d64-4a12-a3e4-bc1534794f30","locationId":"caee6aa1-192c-41a8-96e7-32e511035d13","locationName":"Seattle, WA (Hybrid)","employmentType":"FullTime","secondaryLocations":[{"locationId":"2a20cba2-86e6-4149-a86f-c6e0cbf7dc8a","locationName":"Europe (Remote)","__typename":"JobPostingSecondaryLocation"}]

In [28]:
%%time
try:    
    data = decode(json.dumps(response_json['data']['jobBoard']['jobPostings']), type=list[Posting])
except TypeError as te:
    logger.error(f"An error occurred for company '{ashby_company}'. Perhaps you have the wrong Ashby company name")
#secondaryLocations list
all_locations_json = []
for record in data:
    if len(record.secondaryLocations) == 0:
        continue
    for location in record.secondaryLocations:
        location_json = {}
        location_json['opening_id'] = record.id
        location_json['secondary_location_id'] = location.locationId
        location_json['secondary_location_name'] = location.locationName
        all_locations_json.append(location_json)

#postings list
all_flattened_json = []
for record in data:
    flattened_json_record = {}
    flattened_json_record["opening_id"] = record.id
    flattened_json_record["opening_name"] = record.title
    flattened_json_record["department_id"] = record.teamId
    flattened_json_record["location_id"] = record.locationId
    flattened_json_record["location"] = record.locationName
    flattened_json_record["employment_type"] = record.employmentType
    flattened_json_record["compensation_tier"] = record.compensationTierSummary
    # flattened_json_record["secondary_location_ids"] = all_locations_json[i]['secondary_location_ids']
    # flattened_json_record["secondary_location_names"] = all_locations_json[i]['secondary_location_names']
    all_flattened_json.append(flattened_json_record)
    
#DuckDB calls
ashby_postings = pl.DataFrame(all_flattened_json)
ashby_locations = pl.DataFrame(all_locations_json)

CPU times: user 1.14 ms, sys: 4.63 ms, total: 5.77 ms
Wall time: 14 ms


In [29]:
try:    
    data = decode(json.dumps(response_json['data']['jobBoard']['teams']), type=list[Team])
except TypeError as te:
    logger.error(f"An error occurred for company '{ashby_company}'. Perhaps you have the wrong Ashby company name")

all_teams_json = []
for record in data:
    teams_json_record = {}
    teams_json_record["department_id"] = record.id
    teams_json_record["department_name"] = record.name
    teams_json_record["parent_department_id"] = record.parentTeamId
    all_teams_json.append(teams_json_record)

ashby_departments = pl.DataFrame(all_teams_json)

In [30]:
s3_json_path = f"requests/ashby/date={current_date_utc}/company={ashby_company}/{ashby_company}-ashby.json"
full_s3_json_path = "s3://" + os.environ.get('RAW_HTML_S3_BUCKET') + "/" + s3_json_path
ashby_job_board_source = f"https://jobs.ashbyhq.com/{ashby_company}"

In [31]:
ashby_postings_final = con.execute("""
    select *, 
        ? as s3_json_path,
        ? as created_at,
        ? as updated_at,
        ? as company_name,
        ? as ashby_job_board_source
    from ashby_postings
""", [
    full_s3_json_path,
    created_at,
    updated_at,
    ashby_company,
    ashby_job_board_source
]).pl()

In [34]:
ashby_departments

department_id,department_name,parent_department_id
str,str,str
"""81c51072-2254-…","""Sales""",null
"""9bc8a9d6-e4a6-…","""Account Execut…","""81c51072-2254-…"
"""39846106-fd45-…","""Customer Succe…",null
"""8a933aa8-e492-…","""Design Service…","""39846106-fd45-…"
"""4e2f9c06-0854-…","""Engineering""",null
"""5d37a8d7-7957-…","""Product Engine…","""4e2f9c06-0854-…"
"""5b313289-2379-…","""Finance""",null
"""c2ecdec4-d0ef-…","""Accounting""","""5b313289-2379-…"
"""4685506f-f8d5-…","""Business Devel…","""81c51072-2254-…"


In [32]:
ashby_postings_final

opening_id,opening_name,department_id,location_id,location,employment_type,compensation_tier,s3_json_path,created_at,updated_at,company_name,ashby_job_board_source
str,str,str,str,str,str,str,str,i32,i32,str,str
"""4e1d19dd-88a3-…","""Account Execut…","""9bc8a9d6-e4a6-…","""b15ff6f9-ef4b-…","""Germany""","""FullTime""",null,"""s3://levergree…",1698203119,1698203119,"""aurorasolar""","""https://jobs.a…"
"""4488b091-917b-…","""Design and Eng…","""8a933aa8-e492-…","""e9f58a64-18c3-…","""United States""","""FullTime""","""$75.5K – $123K…","""s3://levergree…",1698203119,1698203119,"""aurorasolar""","""https://jobs.a…"
"""0982b8ef-6131-…","""Manager, Helio…","""9bc8a9d6-e4a6-…","""e9f58a64-18c3-…","""United States""","""FullTime""","""$144K – $160K …","""s3://levergree…",1698203119,1698203119,"""aurorasolar""","""https://jobs.a…"
"""3659f5ed-5dd8-…","""Principal Soft…","""5d37a8d7-7957-…","""fc4d5144-3c43-…","""Canada""","""FullTime""","""CA$143K – CA$2…","""s3://levergree…",1698203119,1698203119,"""aurorasolar""","""https://jobs.a…"
"""a76847c1-14e1-…","""Senior Account…","""c2ecdec4-d0ef-…","""e9f58a64-18c3-…","""United States""","""FullTime""","""$71.5K – $116K…","""s3://levergree…",1698203119,1698203119,"""aurorasolar""","""https://jobs.a…"
"""23682dc3-d47e-…","""Senior Busines…","""4685506f-f8d5-…","""e9f58a64-18c3-…","""United States""","""FullTime""","""$107K – $152.5…","""s3://levergree…",1698203119,1698203119,"""aurorasolar""","""https://jobs.a…"
"""eb0c2080-4de8-…","""Senior FP&A Ma…","""47a54ff8-9912-…","""eea2befe-0ab3-…","""Toronto""","""FullTime""","""CA$134K – CA$1…","""s3://levergree…",1698203119,1698203119,"""aurorasolar""","""https://jobs.a…"
"""65d4bd3b-f513-…","""Senior Manager…","""7b59a3cd-bedc-…","""e9f58a64-18c3-…","""United States""","""FullTime""","""$142K – $212K …","""s3://levergree…",1698203119,1698203119,"""aurorasolar""","""https://jobs.a…"
"""7108dd94-c96d-…","""(Senior) Softw…","""5d37a8d7-7957-…","""fc4d5144-3c43-…","""Canada""","""FullTime""","""CA$91K – CA$17…","""s3://levergree…",1698203119,1698203119,"""aurorasolar""","""https://jobs.a…"


In [36]:
con.sql(
    """
        select 
            ashby_postings.*, 
            posting_department.department_id, 
            posting_department.department_name, 
            posting_department.parent_department_id as primary_department_id,
            parent_department.department_name as primary_department
        from ashby_postings
        left join ashby_departments posting_department on ashby_postings.department_id = posting_department.department_id
        left join ashby_departments parent_department on posting_department.parent_department_id = parent_department.department_id
        --where ashby_postings.department_id = 'c59aa971-2f48-4076-93dc-356c7ec51292'
    """
).df()

,opening_id,opening_name,department_id,location_id,location,employment_type,compensation_tier,department_id_2,department_name,primary_department_id,primary_department
0,4e1d19dd-88a3-45e7-8c3a-672a56068189,"Account Executive, Helioscope",9bc8a9d6-e4a6-46f0-843d-310876578cf0,b15ff6f9-ef4b-4dfa-b279-b28d77578237,Germany,FullTime,NaN,9bc8a9d6-e4a6-46f0-843d-310876578cf0,Account Executive,81c51072-2254-4ab3-bec9-7a5a722aa1c2,Sales
1,4488b091-917b-46dc-bd50-856e6a68d05a,Design and Engineering Specialist,8a933aa8-e492-44ad-8d22-a9feba57a9d2,e9f58a64-18c3-4f0a-90a1-664a2d2b9a1f,United States,FullTime,$75.5K – $123K • Offers Equity • Offers Bonus ...,8a933aa8-e492-44ad-8d22-a9feba57a9d2,Design Services,39846106-fd45-43b0-8793-172645d843ed,Customer Success
2,0982b8ef-6131-4340-b783-7bd0e59d5f56,"Manager, Helioscope Sales",9bc8a9d6-e4a6-46f0-843d-310876578cf0,e9f58a64-18c3-4f0a-90a1-664a2d2b9a1f,United States,FullTime,$144K – $160K • Offers Equity • Multiple Ranges,9bc8a9d6-e4a6-46f0-843d-310876578cf0,Account Executive,81c51072-2254-4ab3-bec9-7a5a722aa1c2,Sales
3,3659f5ed-5dd8-41ad-a64a-74d433b70ff3,Principal Software Engineer,5d37a8d7-7957-4544-bce8-ba8d38293cb1,fc4d5144-3c43-47a1-886f-89aeb1ad76ab,Canada,FullTime,CA$143K – CA$262K • Offers Equity • Offers Bon...,5d37a8d7-7957-4544-bce8-ba8d38293cb1,Product Engineering,4e2f9c06-0854-4b2e-a6d5-e97643ef8d48,Engineering
4,a76847c1-14e1-465d-9a7d-83c7e8d83bbf,Senior Accountant,c2ecdec4-d0ef-451b-9e81-04ddb1088c27,e9f58a64-18c3-4f0a-90a1-664a2d2b9a1f,United States,FullTime,$71.5K – $116K • Offers Equity • Offers Bonus ...,c2ecdec4-d0ef-451b-9e81-04ddb1088c27,Accounting,5b313289-2379-4c78-acff-98216f3104c0,Finance
5,23682dc3-d47e-466d-b19d-7ddbf4998ea9,Senior Business Development Manager,4685506f-f8d5-4c29-bc68-a57445de700a,e9f58a64-18c3-4f0a-90a1-664a2d2b9a1f,United States,FullTime,$107K – $152.5K • Offers Equity • Offers Bonus...,4685506f-f8d5-4c29-bc68-a57445de700a,Business Development,81c51072-2254-4ab3-bec9-7a5a722aa1c2,Sales
6,eb0c2080-4de8-4524-9ac5-ee71d9a1384b,Senior FP&A Manager,47a54ff8-9912-4d9f-8eb0-e9489389b439,eea2befe-0ab3-46a2-84b7-fc8746b23fbf,Toronto,FullTime,CA$134K – CA$181.4K • Offers Equity • Offers B...,47a54ff8-9912-4d9f-8eb0-e9489389b439,FP&A,5b313289-2379-4c78-acff-98216f3104c0,Finance
7,65d4bd3b-f513-4ad8-bbd7-9e46b331e7ed,"Senior Manager, HRBP",7b59a3cd-bedc-474c-945f-50d02fe05e7e,e9f58a64-18c3-4f0a-90a1-664a2d2b9a1f,United States,FullTime,$142K – $212K • Offers Equity • Offers Bonus •...,7b59a3cd-bedc-474c-945f-50d02fe05e7e,HRBP,892de217-cf70-4a60-bb53-2309621366fb,People
8,7108dd94-c96d-4162-b613-5a057124a004,"(Senior) Software Engineer, Frontend",5d37a8d7-7957-4544-bce8-ba8d38293cb1,fc4d5144-3c43-47a1-886f-89aeb1ad76ab,Canada,FullTime,CA$91K – CA$172K • Offers Equity • Offers Bonu...,5d37a8d7-7957-4544-bce8-ba8d38293cb1,Product Engineering,4e2f9c06-0854-4b2e-a6d5-e97643ef8d48,Engineering
9,d877c6fe-6f06-44f8-afdc-c6b00d8a0847,"(Senior) Software Engineer, Full Stack",5d37a8d7-7957-4544-bce8-ba8d38293cb1,fc4d5144-3c43-47a1-886f-89aeb1ad76ab,Canada,FullTime,CA$91K – CA$172K • Offers Equity • Offers Bonu...,5d37a8d7-7957-4544-bce8-ba8d38293cb1,Product Engineering,4e2f9c06-0854-4b2e-a6d5-e97643ef8d48,Engineering


In [11]:
ddb_postings[ddb_postings.department_id == 'c59aa971-2f48-4076-93dc-356c7ec51292']

,opening_id,opening_name,department_id,location_id,location,employment_type,compensation_tier
25,f9335c4c-0457-401c-90f0-545ce6e8ab39,"Financial Manager, Norway",c59aa971-2f48-4076-93dc-356c7ec51292,c4c17428-464a-41b1-a58e-3d3d5ba9a4b9,Norway,FullTime,None


In [9]:
ashby_departments

,department_id,department_name,parent_department_id
0,392812fd-1422-45a1-90be-4d82fd61d6d5,Sales,None
1,8c27e7fa-f898-4de1-adf6-70c67c1e1c11,Engineering,None
2,3fc1ac8a-169a-4bc2-a825-513af17b3752,Business Control,None
3,7237acf4-a988-4b59-bc60-9e0b65fbed0b,Expansion Operations,None
4,19cc7b56-cbce-441e-8d9f-be0b4d58136e,Onboarding,None
5,4c5bf27a-58d5-4235-abf9-93be9278d92d,Communications,None
6,7aca1995-685d-46f0-8829-418a828f9d2c,Finance,None
7,9867a2c4-89b1-484b-8ed6-d3f839ac0b1e,Customer Success,None
8,be8bc0da-4236-40b4-a90b-12f6c43ba8a8,Customer Support,None
9,0ba66639-8de1-432e-961b-d4f4504e448f,Rev Ops & Enablement,None


In [35]:
# all_locations_json

In [28]:
# tmp = tempfile.NamedTemporaryFile(delete=True)
# tmp_file_path = tmp.name
# tmp.write(json.dumps(response_json).encode('utf-8'))
# ddb_json = con.execute(f"""select * from read_json_auto('{tmp_file_path}')""").fetchall()
# # ddb_json = duckdb.sql(f"""select * from read_json_auto('{tmp_file_path}')""")
# # tmp.close()

In [29]:
# import json

# # filepath = f"{tmp_file_path}+.json"

# with open('dataaaa.json', 'w') as f:
#     json.dump(response_json, f)


# with open('dataaaa.json', 'r') as f:
#     data = json.load(f)

# tmp.close()

In [30]:
ddb_postings.head()

,opening_id,opening_name,department_id,location_id,location,employment_type,compensation_tier
0,f7c4de3c-1d64-42e9-8111-f38e265327d8,Account Executive | Mid-Market,baa9ccf5-274b-43e7-a810-1b50545f0745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,$195.5K – $230K • Offers Equity • Offers Commi...
1,8c004352-f162-4d89-858f-75a649e77ec4,Account Executive | SMB,baa9ccf5-274b-43e7-a810-1b50545f0745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,$153K – $180K • Offers Equity • Offers Commission
2,d84bbf19-572a-499c-9c87-0c154ce85caf,Account Manager | Mid-Market,baa9ccf5-274b-43e7-a810-1b50545f0745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,$183.6K – $216K • Offers Equity • Offers Commi...
3,26e5993f-2008-4d51-9362-74e81c03764a,Account Manager | SMB,baa9ccf5-274b-43e7-a810-1b50545f0745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,$153K – $180K • Offers Equity • Offers Commission
4,7b9be1e2-2658-476b-a4b2-20e3dcf8450e,Channel Partner Manager | Accounting,baa9ccf5-274b-43e7-a810-1b50545f0745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,$153K – $180K • Offers Equity • Offers Commission


In [31]:
ddb_locations.head()

,opening_id,secondary_location_id,secondary_location_name
0,f7c4de3c-1d64-42e9-8111-f38e265327d8,edc884f1-bbc2-4d94-b30d-5ef1bc02db96,Miami
1,f7c4de3c-1d64-42e9-8111-f38e265327d8,8143c122-8a48-430b-a32e-ad1f0e306c17,Remote
2,8c004352-f162-4d89-858f-75a649e77ec4,edc884f1-bbc2-4d94-b30d-5ef1bc02db96,Miami
3,8c004352-f162-4d89-858f-75a649e77ec4,8143c122-8a48-430b-a32e-ad1f0e306c17,Remote
4,d84bbf19-572a-499c-9c87-0c154ce85caf,edc884f1-bbc2-4d94-b30d-5ef1bc02db96,Miami


In [13]:
results_locations.head()

NameError: name 'results_locations' is not defined

In [14]:
%%time



data = json.loads(json.dumps(response_json['data']['jobBoard']['jobPostings']))

# Extract the "people" objects as a DataFrame
postings_df = pd.json_normalize(data)

# Extract the "addresses" objects as a DataFrame
locations_df = pd.json_normalize(data, record_path=['secondaryLocations'], meta='id', record_prefix='address_')

# Join the DataFrames on the join key
# result = people_df.merge(addresses_df, left_index=True, right_index=True)


CPU times: user 7.21 ms, sys: 2.42 ms, total: 9.63 ms
Wall time: 23.8 ms


In [32]:
postings_df

,id,title,teamId,locationId,locationName,employmentType,secondaryLocations,compensationTierSummary,__typename
0,f7c4de3c-1d64-42e9-8111-f38e265327d8,Account Executive | Mid-Market,baa9ccf5-274b-43e7-a810-1b50545f0745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,[{'locationId': 'edc884f1-bbc2-4d94-b30d-5ef1b...,$195.5K – $230K • Offers Equity • Offers Commi...,JobPostingBriefsWithIdsAndTeamId
1,8c004352-f162-4d89-858f-75a649e77ec4,Account Executive | SMB,baa9ccf5-274b-43e7-a810-1b50545f0745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,[{'locationId': 'edc884f1-bbc2-4d94-b30d-5ef1b...,$153K – $180K • Offers Equity • Offers Commission,JobPostingBriefsWithIdsAndTeamId
2,7810446b-ab9d-44ab-af19-0363e50529c9,Account Manager | Growth,baa9ccf5-274b-43e7-a810-1b50545f0745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,[{'locationId': 'edc884f1-bbc2-4d94-b30d-5ef1b...,$99.5K – $117K • Offers Equity • Offers Commis...,JobPostingBriefsWithIdsAndTeamId
3,d84bbf19-572a-499c-9c87-0c154ce85caf,Account Manager | Mid-Market,baa9ccf5-274b-43e7-a810-1b50545f0745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,[{'locationId': 'edc884f1-bbc2-4d94-b30d-5ef1b...,$183.6K – $216K • Offers Equity • Offers Commi...,JobPostingBriefsWithIdsAndTeamId
4,26e5993f-2008-4d51-9362-74e81c03764a,Account Manager | SMB,baa9ccf5-274b-43e7-a810-1b50545f0745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,[{'locationId': 'edc884f1-bbc2-4d94-b30d-5ef1b...,$153K – $180K • Offers Equity • Offers Commission,JobPostingBriefsWithIdsAndTeamId
5,95e55e73-03b3-438b-9754-20a7415f447c,Associate Buyer | Savings,5b7b24e9-4ed0-4f0a-8075-f4a1a72fd76e,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,[{'locationId': 'edc884f1-bbc2-4d94-b30d-5ef1b...,$106.3K – $125K • Offers Equity,JobPostingBriefsWithIdsAndTeamId
6,782fa150-e975-4bd6-9141-76427d4c6433,"Associate, Portfolio Finance",918f335f-fc6a-435f-8e85-ff6059166f97,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,[{'locationId': '8143c122-8a48-430b-a32e-ad1f0...,$106.3K – $125K • Offers Equity,JobPostingBriefsWithIdsAndTeamId
7,7b9be1e2-2658-476b-a4b2-20e3dcf8450e,Channel Partner Manager | Accounting,baa9ccf5-274b-43e7-a810-1b50545f0745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,[{'locationId': 'edc884f1-bbc2-4d94-b30d-5ef1b...,$153K – $180K • Offers Equity • Offers Commission,JobPostingBriefsWithIdsAndTeamId
8,4745807e-82f4-4b1a-857c-dc8dadc73076,Credit Risk Analyst,49465194-5c69-43eb-aef1-aafa68e31745,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,[{'locationId': 'edc884f1-bbc2-4d94-b30d-5ef1b...,$114.8K – $135K • Offers Equity,JobPostingBriefsWithIdsAndTeamId
9,d64609dd-c391-45a2-bfdb-c1bb34e8f93c,Customer Experience | Entry Level,9de0cb75-c0c9-4604-b109-5307e3fc8248,727b4420-37ce-4db8-bd81-5404184bfd04,New York,FullTime,[{'locationId': 'edc884f1-bbc2-4d94-b30d-5ef1b...,None,JobPostingBriefsWithIdsAndTeamId


In [33]:
locations_df

,address_locationId,address_locationName,address___typename,id
0,edc884f1-bbc2-4d94-b30d-5ef1bc02db96,Miami,JobPostingSecondaryLocation,f7c4de3c-1d64-42e9-8111-f38e265327d8
1,8143c122-8a48-430b-a32e-ad1f0e306c17,Remote,JobPostingSecondaryLocation,f7c4de3c-1d64-42e9-8111-f38e265327d8
2,edc884f1-bbc2-4d94-b30d-5ef1bc02db96,Miami,JobPostingSecondaryLocation,8c004352-f162-4d89-858f-75a649e77ec4
3,8143c122-8a48-430b-a32e-ad1f0e306c17,Remote,JobPostingSecondaryLocation,8c004352-f162-4d89-858f-75a649e77ec4
4,edc884f1-bbc2-4d94-b30d-5ef1bc02db96,Miami,JobPostingSecondaryLocation,7810446b-ab9d-44ab-af19-0363e50529c9
...,...,...,...,...
71,8143c122-8a48-430b-a32e-ad1f0e306c17,Remote,JobPostingSecondaryLocation,4e64ab86-4e30-403b-b1b9-41dc052570ce
72,edc884f1-bbc2-4d94-b30d-5ef1bc02db96,Miami,JobPostingSecondaryLocation,6a5108f4-8605-4a67-9446-ea05f197694c
73,8143c122-8a48-430b-a32e-ad1f0e306c17,Remote,JobPostingSecondaryLocation,6a5108f4-8605-4a67-9446-ea05f197694c
74,edc884f1-bbc2-4d94-b30d-5ef1bc02db96,Miami,JobPostingSecondaryLocation,da200390-5a7d-4d7e-9b9d-b7faba24cf27


In [16]:
results.df()

NameError: name 'results' is not defined

In [152]:
pd.json_normalize(all_flattened_json)

,opening_id,opening_name,department_id,location_id,location,employment_type,secondary_location_ids,secondary_location_names
0,0e587316-7063-43e6-a91b-27e083b03271,"Account Executive, Client Development Mid Mar...",392812fd-1422-45a1-90be-4d82fd61d6d5,abee6120-4179-40fa-a117-cc41e81ae1dd,Anywhere (North America),FullTime,None,None
1,a47583f6-c8a4-4f73-8753-f937a8c8e0f3,"Account Executive, Client Development SMB - NAM",392812fd-1422-45a1-90be-4d82fd61d6d5,abee6120-4179-40fa-a117-cc41e81ae1dd,Anywhere (North America),FullTime,None,None
2,cb401cfe-19ec-4e9f-bde9-c7a341657d73,"Account Executive, Enterprise - Global Payroll...",392812fd-1422-45a1-90be-4d82fd61d6d5,ffd15d47-d593-402d-b460-a093e25887f8,EMEA,FullTime,"d2a99326-63b2-431e-8555-48d4078541a6, 7289ee4f...","Netherlands, Belgium, Luxembourg"
3,0b9d8cc9-d31a-43a6-b298-6e5f75f23309,"Account Executive, Enterprise - Global Payroll...",392812fd-1422-45a1-90be-4d82fd61d6d5,ffd15d47-d593-402d-b460-a093e25887f8,EMEA,FullTime,None,None
4,b910e027-6b55-45b4-9859-72b3eb15e900,"Account Executive, Enterprise - Global Payroll...",392812fd-1422-45a1-90be-4d82fd61d6d5,ffd15d47-d593-402d-b460-a093e25887f8,EMEA,FullTime,None,None
...,...,...,...,...,...,...,...,...
193,e23a8f79-be0c-4b47-9066-49a05922766f,Team Lead - Engineering,8c27e7fa-f898-4de1-adf6-70c67c1e1c11,9c552a18-8b44-438f-bf27-9a9890f7ec20,Anywhere (EMEA),FullTime,None,None
194,3e42db55-1d6b-41e2-aee7-587dc5aac90b,Technical Support Specialist - APAC,0303f640-3d06-48fc-b048-1d9442d4c581,2fad93ae-e1bd-4df3-8097-89b8abb626d9,Anywhere (APAC),FullTime,None,None
195,c90fcf78-5846-425e-8f54-9e9d5ef2336b,Technical Support Specialist - LATAM,0303f640-3d06-48fc-b048-1d9442d4c581,28092a8a-289c-4901-bbef-ddc01a258c7f,Anywhere (LATAM),FullTime,None,None
196,47af4e62-34fd-4606-b1e8-8b0a09dc4116,Treasury Team Lead - Americas/LATAM (100% remo...,7aca1995-685d-46f0-8829-418a828f9d2c,9a6a380a-1c69-43f3-8205-b2f6b4ee5626,Uruguay,FullTime,"672d2a36-47b3-4b2b-b0cc-4006bf6683ca, b6a5e4bd...","Mexico, Brazil, Argentina, North America"


In [112]:
data

[Posting(id=UUID('0e587316-7063-43e6-a91b-27e083b03271'), title='Account Executive, Client Development  Mid Market - NAM', teamId=UUID('392812fd-1422-45a1-90be-4d82fd61d6d5'), locationId=UUID('abee6120-4179-40fa-a117-cc41e81ae1dd'), locationName='Anywhere (North America)', employmentType='FullTime', compensationTierSummary=None, secondaryLocations=[]),
 Posting(id=UUID('a47583f6-c8a4-4f73-8753-f937a8c8e0f3'), title='Account Executive, Client Development SMB - NAM ', teamId=UUID('392812fd-1422-45a1-90be-4d82fd61d6d5'), locationId=UUID('abee6120-4179-40fa-a117-cc41e81ae1dd'), locationName='Anywhere (North America)', employmentType='FullTime', compensationTierSummary=None, secondaryLocations=[]),
 Posting(id=UUID('cb401cfe-19ec-4e9f-bde9-c7a341657d73'), title='Account Executive, Enterprise - Global Payroll, Benelux', teamId=UUID('392812fd-1422-45a1-90be-4d82fd61d6d5'), locationId=UUID('ffd15d47-d593-402d-b460-a093e25887f8'), locationName='EMEA', employmentType='FullTime', compensationTier

In [168]:
all_locations_json = []
for record in data:
    location_json = {}
    location_json['opening_id'] = record.id
    secondary_location_ids = "" if len(record.secondaryLocations) > 0 else None
    secondary_location_names = ""
    for location in record.secondaryLocations:
        secondary_location_ids = secondary_location_ids + str(location.locationId) + ', '
        secondary_location_names = secondary_location_names + location.locationName + ', '
    try:
        secondary_location_ids = secondary_location_ids[:-2]
        secondary_location_names = secondary_location_names[:-2]
    except:
        pass
    location_json['secondary_location_ids'] = secondary_location_ids
    location_json['secondary_location_names'] = secondary_location_names
    all_locations_json.append(location_json)
all_locations_json

[{'opening_id': '0e587316-7063-43e6-a91b-27e083b03271',
  'secondary_location_ids': None,
  'secondary_location_names': ''},
 {'opening_id': 'a47583f6-c8a4-4f73-8753-f937a8c8e0f3',
  'secondary_location_ids': None,
  'secondary_location_names': ''},
 {'opening_id': 'cb401cfe-19ec-4e9f-bde9-c7a341657d73',
  'secondary_location_ids': 'd2a99326-63b2-431e-8555-48d4078541a6, 7289ee4f-930f-4192-ba49-981bffc31b1b, b0ae272b-5f97-45b7-977e-bafcb027db16',
  'secondary_location_names': 'Netherlands, Belgium, Luxembourg'},
 {'opening_id': '0b9d8cc9-d31a-43a6-b298-6e5f75f23309',
  'secondary_location_ids': None,
  'secondary_location_names': ''},
 {'opening_id': 'b910e027-6b55-45b4-9859-72b3eb15e900',
  'secondary_location_ids': None,
  'secondary_location_names': ''},
 {'opening_id': '5937acdb-a614-42ff-b725-62858170908f',
  'secondary_location_ids': None,
  'secondary_location_names': ''},
 {'opening_id': 'ac5135d9-552c-4ce6-a1eb-ea8af5dd0412',
  'secondary_location_ids': '561da142-1973-4d4a-b204

In [144]:
user_to_repos

{}

In [86]:
data

[Posting(id=UUID('0e587316-7063-43e6-a91b-27e083b03271'), title='Account Executive, Client Development  Mid Market - NAM', teamId=UUID('392812fd-1422-45a1-90be-4d82fd61d6d5'), locationId=UUID('abee6120-4179-40fa-a117-cc41e81ae1dd'), locationName='Anywhere (North America)', employmentType='FullTime', compensationTierSummary=None, secondaryLocations=[]),
 Posting(id=UUID('a47583f6-c8a4-4f73-8753-f937a8c8e0f3'), title='Account Executive, Client Development SMB - NAM ', teamId=UUID('392812fd-1422-45a1-90be-4d82fd61d6d5'), locationId=UUID('abee6120-4179-40fa-a117-cc41e81ae1dd'), locationName='Anywhere (North America)', employmentType='FullTime', compensationTierSummary=None, secondaryLocations=[]),
 Posting(id=UUID('cb401cfe-19ec-4e9f-bde9-c7a341657d73'), title='Account Executive, Enterprise - Global Payroll, Benelux', teamId=UUID('392812fd-1422-45a1-90be-4d82fd61d6d5'), locationId=UUID('ffd15d47-d593-402d-b460-a093e25887f8'), locationName='EMEA', employmentType='FullTime', compensationTier

In [47]:
user_to_repos

{'petroav': {'petroav/6.828'},
 'rspt': {'rspt/rspt-theme'},
 'izuzero': {'izuzero/xe-module-ajaxboard',
  'izuzero/xe-module-ajaxboard-plugins'},
 'SametSisartenep': {'tj/commander.js', 'visionmedia/debug'},
 'winterbe': {'winterbe/streamjs'},
 'hermanwahyudi': {'hermanwahyudi/selenium'},
 'jdilt': {'jdilt/jdilt.github.io'},
 'sundaymtn': {'sundaymtn/waterline'},
 'zhouzhi2015': {'zhouzhi2015/temp'},
 'petrkutalek': {'petrkutalek/png2pos'},
 'caleb-eades': {'caleb-eades/MinecraftServers'},
 'comcxx11': {'phpsysinfo/phpsysinfo'},
 'Soufien': {'prakhar1989/awesome-courses',
  'wasabeef/awesome-android-libraries'},
 'hcremers': {'ktgw0316/LightZone-l10n-nl'},
 'Dmitry-Me': {'JayXon/tinyxml2', 'leethomason/tinyxml2'},
 'chrisndodge': {'edx/edx-platform'},
 'yhoonkim': {'yhoonkim/GraphBoard'},
 'unicomp21': {'brendan-duncan/dartray', 'dartist/raytracer'},
 'SundeepK': {'SundeepK/Clone'},
 'mevlan': {'mevlan/script'},
 'suneg': {'suneg/dojo_rules'},
 'blang': {'blang/btrfs-initrd'},
 'vatsa